In [1]:
import os 
import json
import pandas as pd 
import traceback

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# my_key = os.getenv("GEMINI_API_KEY")

# from langchain_google_genai import ChatGoogleGenerativeAI

# llm = ChatGoogleGenerativeAI(
#     model="gemini-1.5-pro",
#     api_key = my_key,
#     temperature=0.5
# )

In [4]:
my_key = os.getenv("MISTRAL_API_KEY")

In [5]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    api_key = my_key,
    model="mistral-large-latest",
    temperature=0,
    max_retries=2,
)

In [6]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnableSequence
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback

import PyPDF2


In [7]:
RESPONSE_JSON = {
    "quizzes": [
        {
            "question_number": 1,
            "question": "First generated question here based on {text} and {subject}",
            "options": [
                "A) Option A",
                "B) Option B",
                "C) Option C",
                "D) Option D"
            ],
            "correct_answer": "B) Option B",
            "difficulty": "Medium",
            "explanation": "Explanation for the correct answer."
        },
        {
            "question_number": 2,
            "question": "Second generated question here based on {text} and {subject}",
            "options": [
                "A) Option A",
                "B) Option B",
                "C) Option C",
                "D) Option D"
            ],
            "correct_answer": "C) Option C",
            "difficulty": "Hard",
            "explanation": "Explanation for the correct answer."
        }
    ]
}


In [8]:
TEMPLATE1 = """
You are tasked with creating {number} multiple-choice questions for an educational quiz. 
The topic of each question should be based on {text}. 
The subject is {subject}, and the tone of the questions should be {tone}. 
Ensure that each question effectively tests the user's knowledge, is clear, concise, and does not repeat the answer or options. 
For each question, provide {number} options, only one of which is correct. 
Return all questions and options in the following 
### RESPONSE_JSON 
{response_json}
"""

In [9]:
prompt1 = PromptTemplate(
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
    template=TEMPLATE1
)

In [10]:
chain1 = LLMChain(prompt=prompt1, llm=llm, output_key="quiz",verbose=True)

C:\Users\1dile\AppData\Local\Temp\ipykernel_4724\3724210371.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain1 = LLMChain(prompt=prompt1, llm=llm, output_key="quiz",verbose=True)


In [11]:
TEMPLATE2 = """
You are tasked with evaluating the quiz based on the subject: {subject}. 
Please review the quiz questions and options provided in the {quiz}. 
Ensure that the questions effectively assess the knowledge of the respondents, are clear and concise, and that the options are distinct from each other. 
Provide feedback on the clarity of the questions and the appropriateness of the options given. 
Make suggestions for any improvements needed for better evaluation.
"""

In [12]:
prompt2 = PromptTemplate(
    input_variables=['subject', 'quiz'],
    template=TEMPLATE2
)

In [13]:
chain2 = LLMChain(prompt=prompt2, llm=llm, output_key="review",verbose=True)

In [14]:
combine_chain = SequentialChain(
    chains = [chain1, chain2],
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
    output_variables=["quiz","review"],
    verbose=True
)

In [15]:
file_path = r"D:\HP_Laptop\Project\04_GenAI\MCQ_Generator\data.txt"

In [16]:
with open(file_path,'r') as file:
    TEXT = file.read()

print(TEXT)

Generative artificial intelligence (generative AI, GenAI,[1] or GAI) is a subset of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2] These models often generate output in response to specific prompts.[3][4] Generative AI systems learn the underlying patterns and structures of their training data, enabling them to create new data.[5][6]

Improvements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini and LLaMA, text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney and DALL-E, and text-to-video AI generators such as Sora.[7][8][9][10] Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smaller firms have developed generative AI models.[3][11][12]

Generative AI has uses across a wide range of in

In [17]:
NUMBER = 5
SUBJECT = "Generative AI"
TONE = "Simple"

In [18]:
RES_JSON = json.dumps(RESPONSE_JSON)

In [19]:
with get_openai_callback() as cb:
    response = combine_chain.invoke({
        'text':TEXT, 
        'number':NUMBER,
        'subject': SUBJECT,
        'tone':TONE,
        'response_json': RES_JSON
    })



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are tasked with creating 5 multiple-choice questions for an educational quiz. 
The topic of each question should be based on Generative artificial intelligence (generative AI, GenAI,[1] or GAI) is a subset of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2] These models often generate output in response to specific prompts.[3][4] Generative AI systems learn the underlying patterns and structures of their training data, enabling them to create new data.[5][6]

Improvements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini and LLaMA, text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney and DALL-E, and text-to-video AI gen

In [20]:
response

{'text': 'Generative artificial intelligence (generative AI, GenAI,[1] or GAI) is a subset of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2] These models often generate output in response to specific prompts.[3][4] Generative AI systems learn the underlying patterns and structures of their training data, enabling them to create new data.[5][6]\n\nImprovements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini and LLaMA, text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney and DALL-E, and text-to-video AI generators such as Sora.[7][8][9][10] Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smaller firms have developed generative AI models.[3][11][12]\n\nGenerative AI has uses across a wi

In [21]:
print(f"Total Tokens : {cb.total_tokens}")
print(f"Prompt Tokens : {cb.prompt_tokens}")
print(f"Completion Tokens : {cb.completion_tokens}")
print(f"Total Cost : {cb.total_cost}")

Total Tokens : 3478
Prompt Tokens : 1590
Completion Tokens : 1888
Total Cost : 0.0


In [22]:
quiz = response.get('quiz')
print(type(quiz))
quiz

<class 'str'>


'```json\n{\n  "quizzes": [\n    {\n      "question_number": 1,\n      "question": "What is Generative AI primarily used for?",\n      "options": [\n        "A) Analyzing existing data",\n        "B) Creating new data",\n        "C) Storing data",\n        "D) Deleting data",\n        "E) Sorting data"\n      ],\n      "correct_answer": "B) Creating new data",\n      "difficulty": "Easy",\n      "explanation": "Generative AI is primarily used for creating new data such as text, images, videos, or other forms of data."\n    },\n    {\n      "question_number": 2,\n      "question": "Which of the following is an example of a generative AI model?",\n      "options": [\n        "A) Excel",\n        "B) ChatGPT",\n        "C) Google Maps",\n        "D) Microsoft Word",\n        "E) Adobe Photoshop"\n      ],\n      "correct_answer": "B) ChatGPT",\n      "difficulty": "Easy",\n      "explanation": "ChatGPT is an example of a generative AI model that can produce text in response to specific pr

In [23]:
quiz = quiz.replace('```', '').strip()
quiz = quiz.lstrip("json\n").strip()  
quiz


'{\n  "quizzes": [\n    {\n      "question_number": 1,\n      "question": "What is Generative AI primarily used for?",\n      "options": [\n        "A) Analyzing existing data",\n        "B) Creating new data",\n        "C) Storing data",\n        "D) Deleting data",\n        "E) Sorting data"\n      ],\n      "correct_answer": "B) Creating new data",\n      "difficulty": "Easy",\n      "explanation": "Generative AI is primarily used for creating new data such as text, images, videos, or other forms of data."\n    },\n    {\n      "question_number": 2,\n      "question": "Which of the following is an example of a generative AI model?",\n      "options": [\n        "A) Excel",\n        "B) ChatGPT",\n        "C) Google Maps",\n        "D) Microsoft Word",\n        "E) Adobe Photoshop"\n      ],\n      "correct_answer": "B) ChatGPT",\n      "difficulty": "Easy",\n      "explanation": "ChatGPT is an example of a generative AI model that can produce text in response to specific prompts."\n

In [24]:
quiz_dict = json.loads(quiz)
print(type(quiz_dict))
quiz_dict

<class 'dict'>


{'quizzes': [{'question_number': 1,
   'question': 'What is Generative AI primarily used for?',
   'options': ['A) Analyzing existing data',
    'B) Creating new data',
    'C) Storing data',
    'D) Deleting data',
    'E) Sorting data'],
   'correct_answer': 'B) Creating new data',
   'difficulty': 'Easy',
   'explanation': 'Generative AI is primarily used for creating new data such as text, images, videos, or other forms of data.'},
  {'question_number': 2,
   'question': 'Which of the following is an example of a generative AI model?',
   'options': ['A) Excel',
    'B) ChatGPT',
    'C) Google Maps',
    'D) Microsoft Word',
    'E) Adobe Photoshop'],
   'correct_answer': 'B) ChatGPT',
   'difficulty': 'Easy',
   'explanation': 'ChatGPT is an example of a generative AI model that can produce text in response to specific prompts.'},
  {'question_number': 3,
   'question': 'What type of neural network is commonly used in generative AI systems?',
   'options': ['A) Convolutional Neur

In [25]:
quiz_data = []

for value in quiz_dict['quizzes']:
    mcq = value['question']
    options = " | ".join(value['options'])
    correct = value['correct_answer']
    
    quiz_data.append({
        "MCQ": mcq,
        "Choices": options,
        "Correct": correct
    })

In [26]:
quiz_data

[{'MCQ': 'What is Generative AI primarily used for?',
  'Choices': 'A) Analyzing existing data | B) Creating new data | C) Storing data | D) Deleting data | E) Sorting data',
  'Correct': 'B) Creating new data'},
 {'MCQ': 'Which of the following is an example of a generative AI model?',
  'Choices': 'A) Excel | B) ChatGPT | C) Google Maps | D) Microsoft Word | E) Adobe Photoshop',
  'Correct': 'B) ChatGPT'},
 {'MCQ': 'What type of neural network is commonly used in generative AI systems?',
  'Choices': 'A) Convolutional Neural Networks (CNNs) | B) Recurrent Neural Networks (RNNs) | C) Transformer-based Neural Networks | D) Feedforward Neural Networks | E) Radial Basis Function Networks',
  'Correct': 'C) Transformer-based Neural Networks'},
 {'MCQ': 'Which industry has raised concerns about the potential misuse of generative AI?',
  'Choices': 'A) Agriculture | B) Manufacturing | C) Cybersecurity | D) Retail | E) Hospitality',
  'Correct': 'C) Cybersecurity'},
 {'MCQ': 'Which company h

In [29]:
quiz_df = pd.DataFrame(quiz_data)
quiz_df

,MCQ,Choices,Correct
0,What is Generative AI primarily used for?,A) Analyzing existing data | B) Creating new d...,B) Creating new data
1,Which of the following is an example of a gene...,A) Excel | B) ChatGPT | C) Google Maps | D) Mi...,B) ChatGPT
2,What type of neural network is commonly used i...,A) Convolutional Neural Networks (CNNs) | B) R...,C) Transformer-based Neural Networks
3,Which industry has raised concerns about the p...,A) Agriculture | B) Manufacturing | C) Cyberse...,C) Cybersecurity
4,Which company has developed generative AI mode...,A) Apple | B) Amazon | C) OpenAI | D) IBM | E)...,C) OpenAI


In [30]:
quiz_df.to_csv("quiz_data.csv",index=False)